In [6]:
#Other Preprocessing parameters
import cv2
import numpy as np
import os
from pathlib import Path

# Define input and output directories
input_dir = Path("/media/user/data/Mano/Mano-Project/Project-Mano/VISEM-Tracking/VISEM_Tracking_Train_v4/Train/82/images")
output_dir = Path("/media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82")

# Ensure output directory exists
output_dir.mkdir(parents=True, exist_ok=True)

def preprocess_image(image_path):
    """ Preprocess image with Denoising, CLAHE, and Sharpening """
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"Skipping {image_path}, file not found.")
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Step 1: Denoising
    denoised = cv2.fastNlMeansDenoising(gray, None, h=10, templateWindowSize=7, searchWindowSize=21)

    # Step 2: CLAHE
    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(8, 8))
    clahe_img = clahe.apply(denoised)

    # Step 3: Sharpening
    sharpening_kernel = np.array([[0, -1, 0], 
                                  [-1, 5, -1], 
                                  [0, -1, 0]])  
    sharpened = cv2.filter2D(clahe_img, -1, sharpening_kernel)

    return sharpened

# Process all images in the input directory
for image_file in input_dir.glob("*.jpg"):  # Adjust file extension if needed
    processed_img = preprocess_image(image_file)

    if processed_img is not None:
        output_path = output_dir / image_file.name  # Save with the same filename
        cv2.imwrite(str(output_path), processed_img)
        print(f"Processed and saved: {output_path}")

print("Batch Processing Completed!")


Processed and saved: /media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82/82_frame_775.jpg
Processed and saved: /media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82/82_frame_351.jpg
Processed and saved: /media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82/82_frame_1268.jpg
Processed and saved: /media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82/82_frame_611.jpg
Processed and saved: /media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82/82_frame_244.jpg
Processed and saved: /media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82/82_frame_627.jpg
Processed and saved: /media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82/82_frame_1078.jpg
Processed and saved: /media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82/82_frame_54.jpg
Processed and saved: /media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82/82_frame_480.jpg
Processed and save

In [5]:
#Finalized Preprocessing Script
import cv2
import numpy as np
import os
from pathlib import Path

# Define input and output directories
input_dir = Path("/media/user/data/Mano/Mano-Project/Project-Mano/VISEM-Tracking/VISEM_Tracking_Train_v4/Train/82/images")
output_dir = Path("/media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82")

# Ensure output directory exists
output_dir.mkdir(parents=True, exist_ok=True)

def preprocess_image(image_path):
    """ Preprocess image with Bilateral Filtering, CLAHE, and Edge-Preserving Sharpening """
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"Skipping {image_path}, file not found.")
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # **Step 1. Denoising with Bilateral Filter** (Keeps edges sharp, unlike fastNlMeansDenoising)
    denoised = cv2.bilateralFilter(gray, d=9, sigmaColor=75, sigmaSpace=75)
    # d: Diameter of pixel neighborhood
    # sigmaColor: Larger values mean more smoothing
    # sigmaSpace: Higher value preserves edges better

    # **Step 2. CLAHE (Contrast Enhancement)**
    clahe = cv2.createCLAHE(clipLimit=0.8, tileGridSize=(4, 4))  
    # Reduced clipLimit to prevent over-enhancement
    clahe_img = clahe.apply(denoised)

    # **Step 3. Edge-Preserving Sharpening**
    sharpening_kernel = np.array([[0, -1, 0], 
                                  [-1, 5, -1], 
                                  [0, -1, 0]])  # Keeps fine details sharper
    sharpened = cv2.filter2D(clahe_img, -1, sharpening_kernel)

    return sharpened

# Process all images in the input directory
for image_file in input_dir.glob("*.jpg"):  
    processed_img = preprocess_image(image_file)

    if processed_img is not None:
        output_path = output_dir / image_file.name  
        cv2.imwrite(str(output_path), processed_img)
        print(f"Processed and saved: {output_path}")

print("Batch Processing Completed!")


Processed and saved: /media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82/82_frame_775.jpg
Processed and saved: /media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82/82_frame_351.jpg
Processed and saved: /media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82/82_frame_1268.jpg
Processed and saved: /media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82/82_frame_611.jpg
Processed and saved: /media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82/82_frame_244.jpg
Processed and saved: /media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82/82_frame_627.jpg
Processed and saved: /media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82/82_frame_1078.jpg
Processed and saved: /media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82/82_frame_54.jpg
Processed and saved: /media/user/data/Mano/Mano-Project/Project-Mano/Image Preprocessing/82/82_frame_480.jpg
Processed and save

In [9]:
#Video Generation
import cv2
import os

def create_video_from_images(input_folder, output_folder, duration):
    # Define the output video path
    output_video_path = os.path.join(output_folder, "82.mp4")

    # Get all image filenames sorted by frame order
    image_files = sorted(
        [f for f in os.listdir(input_folder) if f.startswith("82_frame_") and f.endswith((".png", ".jpg", ".jpeg"))],
        key=lambda x: int(x.split('_')[-1].split('.')[0])  # Extract frame number for sorting
    )

    if not image_files:
        raise ValueError("No images found in the input folder with the specified naming convention.")

    # Read the first image to get video dimensions
    first_image_path = os.path.join(input_folder, image_files[0])
    first_image = cv2.imread(first_image_path)
    height, width, _ = first_image.shape

    # Calculate frame rate based on total frames and desired duration
    total_frames = len(image_files)
    frame_rate = total_frames / duration

    # Initialize VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4 format
    video_writer = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))

    # Write each image to the video
    for image_file in image_files:
        image_path = os.path.join(input_folder, image_file)
        img = cv2.imread(image_path)
        if img is None:
            print(f"Warning: Unable to read {image_file}. Skipping.")
            continue
        video_writer.write(img)

    # Release the VideoWriter
    video_writer.release()
    print(f"Video saved at: {output_video_path}")

# Input and output paths
input_dir = "/media/user/data/Mano/Mano-Project/Project-Mano/Image_Preprocessing/82/images"
output_dir = "/media/user/data/Mano/Mano-Project/Project-Mano/Image_Preprocessing/82"
video_duration = 30  # Desired duration in seconds

# Create the video
create_video_from_images(input_dir, output_dir, video_duration)

Video saved at: /media/user/data/Mano/Mano-Project/Project-Mano/Image_Preprocessing/82/82.mp4
